In [1]:
import os

# 🔁 Replace this with the name of your JSON key file
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "translation-thesis-466613-eae8275bd604.json"

In [2]:
from google.cloud import translate_v2 as translate
from newspaper import Article
from fpdf import FPDF
import re
import pandas as pd

In [3]:
client = translate.Client()

In [4]:
urls = ["https://www.aljazeera.net/encyclopedia/2009/11/30/%D8%A7%D9%84%D9%88%D8%AB%D9%8A%D9%82%D8%A9-%D8%A7%D9%84%D8%B3%D9%8A%D8%A7%D8%B3%D9%8A%D8%A9-%D9%84%D8%AD%D8%B2%D8%A8-%D8%A7%D9%84%D9%84%D9%87"]

In [5]:
records = []

In [6]:
def safe_filename(text):
    return re.sub(r'[\\/*?:"<>|]', "", text)

In [4]:
import re

def split_text_arabic(text, max_chunk_size=2000):
    # Split by Arabic sentence endings
    sentences = re.split(r'(?<=[\.؟!…])\s*', text)
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) < max_chunk_size:
            current_chunk += sentence + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
    
    if current_chunk.strip():
        chunks.append(current_chunk.strip())
    
    return chunks


In [5]:
def translate_chunks(chunks, source='ar', target='en', batch_size=10):
    translated = []
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]
        print(f"Translating batch {i // batch_size + 1} of {len(chunks) // batch_size + 1}...")
        result = client.translate(batch, source_language=source, target_language=target)
        translated.extend([r['translatedText'] for r in result])
    return translated


In [9]:
for i, url in enumerate(urls):
    try:
        article = Article(url, language='ar')
        article.download()
        article.parse()
        print('parsing')

        title = article.title or f"Article {i+1}"
        text = article.text or ""
        print('title:', title, 'text:', text[:100])

        title_chunks = split_text_arabic(title)
        body_chunks = split_text_arabic(text)
        print('Number of title chunks:', len(title_chunks), 'Number of body chunks:', len(body_chunks))

        print("Sending first translation request...")
        translated_title = translate_chunks(title_chunks, batch_size=10)
        print("Finished translating title.")
        translated_body = translate_chunks(body_chunks, batch_size=16)

        print('Translated title:', translated_title)

        records.append({
            "filename": f"article_{i+1}.pdf",
            "url": url,
            "title_translated": " ".join(translated_title),
            "text_translated": "\n\n".join(translated_body)
        })

        print(f"✅ Translated article {i+1}")

    except Exception as e:
        print(f"❌ Failed article {i+1}: {e}")


parsing
title: أرواح غزة تتساقط تحت مقصلة التجويع وتحذيرات من موت جماعي text: تواصل المجاعة حصد الأرواح في غزة وبات الجوع ينهش الأجساد جنبا إلى جنب مع نيران الاحتلال الإسرائيلي ا
Number of title chunks: 1 Number of body chunks: 2
Sending first translation request...
Translating batch 1 of 1...
Finished translating title.
Translating batch 1 of 1...
Translated title: ['Gaza&#39;s lives are falling under the guillotine of starvation, and there are warnings of mass death.']
✅ Translated article 1


In [10]:
df = pd.DataFrame(records)
df.to_csv("translated_articles.csv", index=False)
print(df.head())

        filename                                                url  \
0  article_1.pdf  https://www.aljazeera.net/news/2025/7/22/%D8%B...   

                                    title_translated  \
0  Gaza&#39;s lives are falling under the guillot...   

                                     text_translated  
0  Famine continues to claim lives in Gaza, and h...  


In [11]:
df = pd.read_csv("translated_articles.csv")

for i, row in df.iterrows():
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)

    pdf.set_font("Arial", style='B', size=14)
    pdf.multi_cell(0, 10, row['title_translated'])
    pdf.ln()

    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 10, row['text_translated'])

    pdf.output(row['filename'])
    print(f"📝 Saved PDF: {row['filename']}")


📝 Saved PDF: article_1.pdf


## For Manual Copy Paste

In [4]:
arabic_title = 'نص الوثيقة السياسية لحزب الله'

In [5]:
arabic_text = """بعد الانتصارات العسكرية للمقاومة اللبنانية ضد إسرائيل، قدم حسن الأمين العام لحزب الله نصر الله الوثيقة السياسية الجديدة لحزبه في خطاب ألقاه في نوفمبر /تشرين الثاني 2009، وحدد فيه الخطوط العريضة لسياسة الحزب فيما يتعلق بعدد من القضايا الداخلية والخارجية. وفي ما يلي النص الكامل للوثيقة: تهدف هذه الوثيقة إلى تظهير الرؤية السياسية لحزب الله، حيث تنطوي على ما نراه من تصورات ومواقف وما نختزنه من آمال وطموحات وهواجس، وهي تأتي – قبل أي شيء آخر – نتاجاً لما خبرناه جيداً من أولوية الفعل وأسبقية التضحية.ففي مرحلة سياسية إستثنائية وحافلة بالتحولات لم يعد ممكناً مقاربة تلك التحولات من دون ملاحظة المكانة الخاصة التي باتت تشغلها مقاومتنا، أو تلك الرزمة من الإنجازات التي حققتها مسيرتنا.وسيكون ضرورياً إدراج تلك التحولات في سياق المقارنة بين مسارين متناقضين وما بينهما من تناسب عكسي متنامٍ:1- مسار المقاومة والممانعة في طوره التصاعدي الذي يستند إلى انتصارات عسكرية ونجاحات سياسية وترسُّخ أنموذج المقاومة شعبياً وسياسياً والثبات في المواقع والمواقف السياسية رغم ضخامة الإستهداف وجسامة التحديات.. وصولاً إلى إمالة موازين القوى في المعادلة الإقليمية لصالح المقاومة وداعميها.2- مسار التسلط والإستكبار الأميركي – الإسرائيلي بأبعاده المختلفة وتحالفاته وامتداداته المباشرة وغير المباشرة والذي يشهد انكسارات أو انهزامات عسكرية وإخفاقات سياسية أظهرت فشلاً متلاحقاً للإستراتيجيات الأميركية ومشاريعها واحداً تلو الآخر، كل ذلك أفضى إلى حالة من التخبط والتراجع والعجز في القدرة على التحكم في مسار التطورات والأحداث في عالمنا العربي والإسلامي.تتكامل هذه المعطيات في إطار مشهد دُوليّ أوسع، يُسهم بدوره في كشف المأزق الأميركي وتراجع هيمنة القطب الواحد لصالح تعددية لم تستقر ملامحها بعد.وما يعمّق أزمة النظام الإستكباري العالمي الإنهيارات في الأسواق المالية الأميركية والعالمية ودخول الإقتصاد الأميركي في حالة تخبّط وعجز، والتي تعبِّر عن ذروة تفاقم المأزق البنيوي في الأنموذج الرأسمالي المتغطرس.
لذا يمكن القول: إننا في سياق تحولات تاريخية تُنذر بتراجع الولايات المتحدة الأميركية كقوة مهيمنة، وتحلُّل نظام القطب الواحد المهيمن، وبداية تشكّل مسار الأفول التاريخي المتسارع للكيان الصهيوني.تقف حركات المقاومة في صُلب هذه التحولات، وتَبرز كمعطى إستراتيجي أساسي في هذا المشهد الدُّولي بعد أن أدت دوراً مركزياً في إنتاج أو تحفيز ما يتصل من تلك التحولات بمنطقتنا.لقد كانت المقاومة في لبنان ومن ضمنها مقاومتنا الإسلامية سبّاقةً إلى مواجهة الهيمنة والإحتلال قبل ما يزيد على عقدين ونصف من الزمن، وهي تمسكت بهذا الخيار في وقت بدا وكأنه تدشينٌ للعصر الأميركي الذي جرت محاولات تصويره وكأنه نهاية للتاريخ. وفي ظل موازين القوى والظروف السائدة آنذاك إعتبر البعض خيار المقاومة وكأنه ضربُ وهمٍ، أو تهورٌ سياسي أو جنوح مناقض لموجبات العقلانية والواقعية.رغم ذلك مضت المقاومة في مسيرتها الجهادية وهي على يقين من أحقية قضيتها وقدرتها على صنع الإنتصار، من خلال الإيمان بالله تعالى والتوكل عليه والإنتماء للأمة جمعاء والإلتزام بالمصالح الوطنية اللبنانية والثقة بشعبها وإعلائها القيم الإنسانية في الحق والعدالة والحرية.فعلى مدى مسارها الجهادي الطويل وعبر انتصاراتها الموصوفة، بدءاً من دحر الإحتلال الإسرائيلي في بيروت والجبل إلى هروبه من صيدا وصور والنبطية وعدوان تموز 1993 وعدوان نيسان 1996 والتحرير في أيار 2000، فحرب تموز 2006، أرست هذه المقاومة صدقيتها وأنموذجها قبل أن تصنع انتصاراتها، فراكمت حقبات تطوُّر مشروعِها من قوة تحرير إلى قوة توازن ومواجهة ومن ثَمّ إلى قوة ردعٍ ودفاع، مضافاً إلى دورها السياسي الداخلي كركن مؤثّر في بناء الدولة القادرة والعادلة.بالتزامن مع ذلك، قُدِّر للمقاومة أن تطور مكانتها السياسية والإنسانية، فارتقت من كونها قيمةً وطنيةً لبنانيةً إلى كونها – أيضاً – قيمةً عربيةً وإسلاميةً متألقةً، وقد أصبحت اليوم قيمةً عالميةً وإنسانيةً يجري استلهام أنموذجها والبناء على إنجازاتها في تجارب وأدبيات كل الساعين إلى من أجل الحرية والإستقلال في شتى أنحاء المعمورة.إنّ حزب الله رغم إدراكه لتلك التحولات الواعدة وما يراه من مراوحة العدو بين عجز استراتيجية الحرب لديه والعجز عن فرض التسويات بشروطه، فإنه لا يستهين بحجم التحديات والمخاطر التي لا تزال ماثلةً، ولا يُقَلِّل من وعورة مسار المواجهة وحجم التضحيات التي تستوجبها مسيرة المقاومة واسترداد الحقوق والمساهمة في استنهاض الأمة، إلاّ أنه – في قبال ذلك – قد بات أشد وضوحاً في خياراته وأمضى عزيمةً في إرادته وأكثر ثقةً بربّه ونفسه وشعبه.
في هذا السياق، يحدِّد حزب الله الخطوط الأساسية التي تشكّل إطاراً فكرياً – سياسياً لرؤيته ومواقفه تجاه التحديات المطروحة.(الفصل الأول): الهيمنة والإستنهاض أولاً : العالم والهيمنة الغربية والأميركيةبعد الحرب العالمية الثانية باتت الولايات المتحدة صاحبة مشروع الهيمنة المركزي والأول، وعلى يديها شهد هذا المشروع تطوراً هائلاً في آليات التسلط والإخضاع غير المسبوقة تاريخياً، مستفيدةً في ذلك من حصيلة مركّبة من الإنجازات المتعددة الأوجه والمستويات العلمية والثقافية والمعرفية والتكنولوجية والإقتصادية والعسكرية، والمدعومة بمشروع سياسي إقتصادي لا ينظر إلى العالم إلاّ بوصفه أسواقاً مفتوحةً ومحكومةً لقوانينها الخاصة. إنّ أخطر ما في منطق الهيمنة الغربي عموماً، والأميركي تحديداً، هو اعتباره منذ الأساس أنه يمتلك العالم وأنّ له حق الهيمنة من منطلق التفوق في أكثر من مجال، ولذا باتت الإستراتيجية التوسعية الغربية – وبخاصة الأميركية – ومع اقترانها بالمشروع الإقتصادي الرأسمالي إستراتيجيةً عالميةَ الطابع، لا حدود لأطماعها وجشعها.إنّ تحكّم قوى الرأسمالية المتوحشة، المتمثلة على نحوٍ رئيسٍ بشبكات الإحتكارات الدُّولية من شركات عابرة للقوميات بل وللقارات، والمؤسسات الدُّولية المتنوعة، وخصوصاً المالية منها والمدعومة بقوة فائقة عسكرياً، أدى الى المزيد من التناقضات والصراعات الجذرية، ليس أقلها اليوم: صراعات الهويات والثقافات وأنماط الحضارات، إلى جانب صراعات الغنى والفقر.لقد حولت الرأسمالية المتوحشة العولمة إلى آلية لبث التفرقة وزرع الشقاق وتدمير الهويات وفرض أخطر أنواع الإستلاب الثقافي والحضاري والإقتصادي والإجتماعي.وقد بلغت العولمة حدَّها الأخطر مع تحولها إلى عولمة عسكرية على أيدي حاملي مشروع الهيمنة الغربي، والتي شهدنا أكثر تعبيراتها في منطقة الشرق الأوسط بدءاً من أفغانستان إلى العراق وفلسطين فلبنان، الذي كان نصيبه منها عدواناً شاملاً في تموز العام 2006 وذلك باليد الإسرائيلية.لم يبلغ مشروع الهيمنة والتسلط الأميركي مستويات خطرةً كما بلغها مؤخراً، لا سيما منذ العقد الأخير من القرن العشرين وحتى اليوم، وذلك في مسار تصاعدي إتخذ من سقوط الإتحاد السوفياتي وتفككه نقطة انطلاق، لما شكّله من فرصة تاريخية في الحسابات الأميركية للإستفراد بقيادة مشروع الهيمنة عالمياً، وذلك باسم المسؤولية التاريخية وبأنّ لا تمييز بين مصلحة العالم والمصلحة الأميركية، ما يعني تسويق الهيمنة كمصلحة لباقي الدول والشعوب لا بوصفها مصلحةً أميركيةً بحتةً.لقد بلغ هذا المسار ذروته مع إمساك تيار المحافظين الجدد بمفاصل إدارة "بوش" الابن، هذا التيار الذي عبّر عن رؤاه الخاصة من خلال وثيقة "مشروع القرن الأميركي الجديد" التي كُتبت ما قبل إجراء الإنتخابات الأميركية عام 2000، ليجد المشروع طريقه إلى التنفيذ بُعيد استلام إدارة "بوش" الابن السلطة في الولايات المتحدة الأميركية.لم يكن غريباً ولا مفاجئاً أن يكون أكثر ما أكدت عليه هذه الوثيقة – التي سرعان ما باتت دليل العمل لإدارة "بوش"  – مسألة إعادة بناء القدرات الأميركية، والتي عكست رؤيةً استراتيجيةً جديدةً للأمن القومي الأميركي، بدا واضحاً تماماً أنها ترتكز على بناء قدرات عسكرية ليس باعتبارها قوة ردع فقط وإنما أيضاً باعتبارها قوة فعل وتدخّل، سواء للقيام بعمليات "وقائية" عن طريق توجيه ضربات إستباقية أم لأغراض علاجية من خلال التعامل مع الأزمات بعد وقوعها. وجدت إدارة "بوش" بُعيد أحداث 11 أيلول 2001، أنها أمام فرصة سانحة لممارسة أكبر قَدْر من النفوذ والتأثير، من خلال وضع رؤيتها الإستراتيجية للهيمنة المنفردة على العالم موضع التطبيق تحت شعار "الحرب الكونية على الإرهاب" وهكذا قامت هذه الإدارة بمحاولات إعتُبرت ناجحةً في بدايتها وفق التالي:1- عسكرة علاقاتها وسياساتها الخارجية إلى الحد الأقصى.
2- تجنّب الإعتماد على الأطر المتعددة الأطراف، والإنفراد باتخاذ القرارات الإستراتيجية، والتنسيق حيث هناك ضرورة، ومع حلفاء يمكن الركون إليهم.
3- حسم الحرب في أفغانستان بسرعة للتفرغ بعد ذلك للخطوة التالية والأهم في مشروع الهيمنة ألا وهي: السيطرة على العراق، الذي اعتُبر نقطة الإرتكاز الرئيسة لإقامة شرق أوسط جديد يتناسب مع متطلبات عالم ما بعد 11 أيلول. ولم تتورع هذه الإدارة عن اللجوء إلى كل أساليب التمويه والخداع والكذب الصريح لتبرير حروبها، لا سيما الحرب على العراق، وضد كل مَن يقاوم مشروعَها الإستعماري الجديد من دول وحركات وقوى وشخصيات.  وفي هذا الإطار عمدت هذه الإدارة إلى إقامة تطابق بين "مقولة الإرهاب" و"مقولة المقاومة" لتنزع عن المقاومة شرعيتها الإنسانية والحقوقية، وتبرّر بالتالي خوض الحروب على أنواعها ضدها، في سياق إزالة آخر حصون دفاع الشعوب والدول عن حقها بالعيش بحرية وكرامة وعزة، وعن حقها بسيادة غير منقوصة وببناء تجاربها الخاصة وأخذ مواقعها وأدوارها في حركة التاريخ الإنساني حضارياً وثقافياً. لقد تحوّل عنوان "الإرهاب" إلى ذريعة أميركية للهيمنة من خلال أدوات: الملاحقة/ الضبط والإعتقال التعسفي/ إفتقاد أبسط مقومات المحاكمات العادلة، كما نجدها في معتقلات "غوانتنامو"، ومن خلال التدخل المباشر في سيادة الدول وتحويلها إلى ماركة مسجلة للتجريم التعسفي واتخاذ إجراءات معاقَبة تطال شعوباً بأسرها، وصولاً إلى منح نفسها حقاً مطلقاً بشن حروب تدميرية وماحقة لا تميز بين البريء والمجرم، ولا بين الطفل والشيخ والمرأة والشاب.لقد بلغت كلفة حروب الإرهاب الأميركية حتى الآن الملايين من البشر فضلاً عن مظاهر الدمار الشاملة، التي لم تُصِب الحجر والبنى التحتية فحسب وإنما بنية ومكونات المجتمعات نفسها، حيث جرى تفكيكها، وعَكْس مسار تطورها التاريخي، في عملية ارتكاس أعادت إنتاج صراعات أهلية ذات أبعاد مذهبية وطائفية وعرقية لا تنتهي. هذا من غير أن ننسى استهداف المخزون الثقافي والحضاري لهذه الشعوب.لا شك أنّ الإرهاب الأميركي هو أصل كل إرهاب في العالم، وقد حولت إدارة "بوش" الولايات المتحدة إلى خطر يتهدد العالم بأسره على كل الصعد والمستويات. ولو جرى اليوم استطلاع عالمي للرأي لظهرت الولايات المتحدة كأكثر دولة مكروهة في العالم.إنّ الإخفاق الذي مُنيت به الحرب على العراق وتطور حالة المقاومة فيه والإمتعاض الإقليمي والدُّولي من نتائج هذه الحرب، وفشل ما يسمى "الحرب على الإرهاب" خصوصاً في أفغانستان إلى تسجيل عودة قوية لحركة طالبان واعتراف بدورها والسعي لعقد تسويات معها، وكذلك الفشل الذريع للحرب الأميركية على المقاومة في لبنان وفلسطين بأدوات إسرائيلية، أدت إلى تآكل الهيبة الأميركية دُوليّاً وإلى تراجع استراتيجي في قدرة الولايات المتحدة على الفعل أو خوض المغامرات الجديدة.إلاّ أنّ ما تقدَّم لا يعني بأنّ الولايات المتحدة ستُخْلي الساحة بسهولة بل ستقوم بكل ما يلزم من أجل حماية ما تسمّيه "مصالحها الإستراتيجية"، ذلك لأنّ سياسات الهيمنة الأميركية تنهض على اعتبارات إيديولوجية ومشاريع فكرية تغذّيها اتجاهاتٌ متطرفةٌ متحالفةٌ مع مركّب صناعي – عسكري لا حدود لجشعه وأطماعه.ثانياً : منطقتنا والمشروع الأميركيإذا كان العالم المستضعَف بأسره يرزح تحت نير هذه الهيمنة الإستكبارية، فإنّ عالمنا العربي والإسلامي يناله منها القسط الأوفر والأثقل، لدواعي تاريخه وحضارته وموارده وموقعه الجغرافي.إنّ عالمنا العربي والإسلامي هو منذ قرون عرضةً لحروب إستعمارية وحشية لا تنتهي، إلاّ أنّ مراحلها الأكثر تقدماً بدأت مع زرع الكيان الصهيوني في المنطقة، ومن ضمن مشروع تفتيتها إلى كيانات متصارعة ومتنابذة تحت عناوين شتى. ولقد بلغت ذروة هذه المرحلة الإستعمارية مع وراثة الولايات المتحدة للإستعمار القديم في المنطقة.يتمثل الهدف المركزي الأبرز للإستكبار الأميركي بالسيطرة على الشعوب بكل أشكالها: السياسية والإقتصادية والثقافية ونهب ثرواتها ويأتي في الطليعة نهب الثروة النفطية بما هي أداة رئيسة للتحكم بروح الإقتصاد العالمي، وبكل الأساليب التي لا تلتزم أي ضوابط أو معايير أخلاقية أو إنسانية، بما فيها استخدام القوة العسكرية المفرِطة، مباشرة أو بالواسطة.وقد اعتمدت أميركا لتحقيق هدفها سياسات عامةً واستراتيجيات عمل، أبرزها:1- توفير كل سبل ضمان الإستقرار للكيان الصهيوني، بما هو قاعدة متقدمة ونقطة ارتكاز للمشروع الأميركي الإستعماري والتفتيتي للمنطقة، ودعم هذا الكيان بكل عوامل القوة والإستمرار، وتوفير شبكة أمان لوجوده، ما يؤهله للعب دور الغدة السرطانية التي تستنزف قدرات الأمة وطاقاتها وتبعثر إمكاناتها وتشتت آمالَها وتطلعاتها.2- تقويض الإمكانات الروحية والحضارية والثقافية لشعوبنا والعمل على إضعاف روحها المعنوية عبر بث حروب إعلامية ونفسية تطال قيمها ورموز جهادها ومقاومتها.3- دعم أنظمة التبعية والإستبداد في المنطقة.4- الإمساك بالمواقع الجغرافية الإستراتيجية للمنطقة، لما تشكّله من عقدة وَصْلٍ وفصل، براً وبحراً وجواً، ونشر القواعد العسكرية في مفاصلها الحيوية خدمةً لحروبها ودعماً لأدواتها.5- منع قيام أي نهضة في المنطقة تسمح بامتلاك أسباب القوة والتقدم أو تلعب دوراً تاريخياً على مستوى العالم.6- زرع الفتن والإنقسامات على أنواعها، لا سيما الفتن المذهبية بين المسلمين، لإنتاج صراعات أهلية داخلية لا تنتهي.من الواضح أنه لا مجال اليوم لقراءة أي صراع في أي منطقة من مناطق العالم إلاّ من منظار إستراتيجي عالمي، فالخطر الأميركي ليس خطراً محلياً أو مختصاً بمنطقة دون أخرى، وبالتالي فإنّ جبهة المواجهة لهذا الخطر الأميركي يجب أن تكون عالميةً أيضاً.ولا شك بأنّ هذه المواجهة صعبةٌ ودقيقةٌ، وهي معركةٌ ذات مدًى تاريخي، وهي بالتالي معركة أجيال وتستلزم الإستفادة من كل قوة مفترَضة، ولقد علّمتنا تجربتنا في لبنان أنّ الصعوبة لا تعني الإستحالة بل بالعكس فإنّ شعوباً حيةً ومتفاعِلةً، وقيادةً حكيمةً وواعيةً ومستعدةً لكل الإحتمالات تراهن على تراكم الإنجازات، تصنع النصر تلو النصر. وبقدر ما يصح هذا الأمر عمودياً عبر التاريخ فهو يصح أفقياً بالإمتداد الجغرافي والجيو-سياسي أيضاً.لم يترك الإستكبار الأميركي لأمتنا وشعوبها من خيار إلاّ خيار المقاومة، من أجل حياة أفضل، ومن أجل مستقبل بشري وإنساني أفضل، مستقبل محكوم بعلاقات من الأخوّة والتنوع والتكافل في آن، ويسوده السلام والوئام، تماماً كما رَسَمَت معالِمَه حركةُ الأنبياء والمصلحين العظام عبر التاريخ، وكما هو في تطلعات وأشواق الروح الإنسانية الحقة والمتسامية.(الفصل الثاني)لبـــــــــنانأولاً : الوطــنإنّ لبنان هو وطننا ووطن الآباء والأجداد، كما هو وطن الأبناء والأحفاد وكل الأجيال الآتية، وهو الوطن الذي قدّمنا من أجل سيادته وعزته وكرامته وتحرير أرضه أغلى التضحيات وأعزّ الشهداء. هذا الوطن نريده لكل اللبنانيين على حد سواء، يحتضنهم ويتسع لهم ويشمخ بهم وبعطاءاتهم.ونريده واحداً موحَّداً، أرضاً وشعباً ودولةً ومؤسسات، ونرفض أي شكل من أشكال التقسيم أو "الفدرلة" الصريحة أو المقنَّعة. ونريده سيداً حراً مستقلاً عزيزاً كريماً منيعاً قوياً قادراً، حاضراً في معادلات المنطقة، ومساهماً أساسياً في صنع الحاضر والمستقبل كما كان حاضراً دائماً في صنع التاريخ.ومن أهم الشروط لقيام وطن من هذا النوع واستمراره أن تكون له دولةٌ عادلةٌ وقادرةٌ وقويةٌ، ونظامٌ سياسيٌ يمثّل بحق إرادة الشعب وتطلعاته الى العدالة والحرية والأمن والإستقرار والرفاه والكرامة، وهذا ما ينشده كل اللبنانيين ويعملون من أجل تحقيقه ونحن منهم.ثانياً : المقـاومـةتمثّل "إسرائيل" تهديداً دائماً للبنان – الدولة والكيان – وخطراً داهماً عليه لجهة أطماعها التاريخية في أرضه ومياهه، وبما هو أنموذج لتعايش أتباع الرسالات السماوية، في صيغة فريدة، ووطن نقيض لفكرة الدولة العنصرية التي تتمظهر في الكيان الصهيوني. فضلاً عن ذلك فإنّ وجود لبنان على حدود فلسطين المحتلة، وفي منطقة مضطربة جراء الصراع مع العدو الإسرائيلي، حتّم على هذا البلد تحمّل مسؤوليات وطنيةً وقوميةً.بدأ التهديد الإسرائيلي لهذا الوطن منذ أن زُرع الكيان الصهيوني في أرض فلسطين، وهو كيانٌ لم يتوانَ عن الإفصاح عن أطماعه بأرض لبنان لضم أجزاء منه، والإستيلاء على خيراته وثرواته وفي مقدمتها مياهه، وحاول تحقيق هذه الأطماع تدريجياً.باشر هذا الكيان عدوانه منذ العام 1948، من الحدود إلى عمق الوطن، من مجزرة حولا عام 1949 إلى العدوان على مطار بيروت الدُّولي عام 1968، وما بينهما سنوات طويلة من الإعتداءات على مناطق الحدود، بأرضها وسكانها وثرواتها، كمقدمة للإستيلاء المباشر على الأرض عن طريق الإجتياحات المتكررة، وصولاً إلى اجتياح آذار 1978 واحتلال منطقة الحدود وإخضاعها لسلطته الأمنية والسياسية والإقتصادية في إطار مشروع متكامل، تمهيداً لإخضاع الوطن كلّه في اجتياح العام 1982.كل ذلك كان يجري بدعم كامل من قِبَل الولايات المتحدة الأميركية، وتجاهل إلى حدّ التواطؤ من قِبَل ما يُعرف بـ"المجتمع الدُّولي" ومؤسساته الدُّولية، وفي ظل صمت رسمي عربي مريب, وغياب للسلطة اللبنانية التي تركت أرضَها وشعَبها نهباً للمجازر والإحتلال الإسرائيلي من دون أن تتحمل مسؤلياتها وواجباتها الوطنية.في ظل هذه المأساة الوطنية الكبرى، ومعاناة الشعب وغياب دولته وتخلّي العالم عنه، لم يجد اللبنانيون المخلصون لوطنهم سوى استخدام حقهم، والإنطلاق من واجبهم الوطني والأخلاقي والديني في الدفاع عن أرضهم، فكان خيارهم: إطلاق مقاومة شعبية مسلّحة لمواجهة الخطر الصهيوني والعدوان الدائم على حياتهم وأرزاقهم ومستقبلهم.في تلك الظروف الصعبة، حيث افتقد اللبنانيون الدولة بدأت مسيرة استعادة الوطن من خلال المقاومة المسلّحة، وذلك بتحرير الأرض والقرار السياسي من يد الإحتلال الإسرائيلي كمقدمة لاستعادة الدولة وبناء مؤسساتها الدستورية، والأهم من ذلك كلّه إعادة تأسيس القيم الوطنية التي يُبنى عليها الوطن وفي طليعتها: السيادة والكرامة الوطنيتان، ما أعطى لقيمة الحرية بُعدَها الحقيقي، فلم تبقَ مجرد شعار معلَّق بل كرّستها المقاومة بفعل تحرير الأرض والإنسان وتحولت هذه القيم الوطنية إلى مدماك أساس لبناء لبنان الحديث، فحجزت موقعه على خارطة العالم وأعادت الإعتبار إليه كبلد يَفرِِض احترامَه ويفتخر أبناؤه بالإنتماء إليه بما هو وطن للحرية والثقافة والعلم والتنوع كما هو وطن العنفوان والكرامة والتضحية والبطولة. إنّ هذه الأبعاد مجتمعةً توّجتها المقاومة من خلال ما أنجزته من تحرير في العام 2000 ومن انتصار تاريخي في حرب تموز عام 2006، والذي قدّمت فيهما تجربةً حيةً للدفاع عن الوطن، تجربةً تحوّلت إلى مدرسة تستفيد منها الشعوب والدول في الدفاع عن أرضها وحماية استقلالها وصون سيادتها.تَحقَّقََ هذا الإنجاز الوطني للمقاومة بمؤازرة شعب وفيّ وجيش وطنيّ، وأَحبطَ أهدافَ العدو وأوقعَ به هزيمةً تاريخيةً، لِتَخْرج المقاومة بمجاهديها وشهدائها ومعها لبنان بشعبه وجيشه بانتصار عظيم أسّس لمرحلة جديدة في المنطقة عنوانها محورية المقاومة دوراً ووظيفةً في ردع العدو وتأمين الحماية لاستقلال الوطن وسيادته والدفاع عن شعبه واستكمال تحرير بقية الأرض المحتلة.إنّ هذا الدور وهذه الوظيفة ضرورةٌ وطنيةٌ دائمةٌ دوام التهديد الإسرائيلي ودوام أطماع العدو في أرضنا ومياهنا ودوام غياب الدولة القوية القادرة، وفي ظل الخلل في موازين القوى ما بين الدولة والعدو – الخلل الذي يدفع عادةً الدول الضعيفة والشعوب المستهدفة من أطماع وتهديدات الدول المتسلطة والقـوية، إلـى البحث عن صيغ تستفيد من القدرات والإمكانات المتاحة – فإنّ التهديد الإسرائيلي الدائم يفرض على لبنان تكريس صيغة دفاعية تقوم على المزاوجة بين وجود مقاومة شعبية تساهم في الدفاع عن الوطن في وجه أي غزو إسرائيلي، وجيش وطني يحمي الوطن ويثبّت أمنه واستقراره، في عملية تكامل أثبتت المرحلة الماضية نجاحها في إدارة الصراع مع العدو وحققت انتصارات للبنان ووفرت سبل الحماية له.هذه الصيغة، التي توضع من ضمن استراتيجية دفاعية، تشكّل مظلّة الحماية للبنان، بعد فشل الرهانات على المظلات الأخرى، سواء أكانت دُوليّة أم عربية أم تفاوضية مع العدو، فانتهاج خيار المقاومة حقّق للبنان تحريراً للأرض واستعادةً لمؤسسات الدولة وحمايةً للسيادة وإنجازاً للإستقلال الحقيقي.  في هذا الإطار فإنّ اللبنانيين بقواهم السياسية وشرائحِهم الإجتماعية ونُخَبِهم الثقافية وهيئاتِهم الإقتصادية، معنيون بالحفاظ على هذه الصيغة والإنخراط فيها، لأنّ الخطر الإسرائيلي يتهدّد لبنان بكل مكوناته ومقوماته، وهذا ما يتطلّب أوسع مشاركة لبنانية في تحمّل مسؤوليات الدفاع عن الوطن وتوفير سبل الحماية له.إنّ نجاح تجربة المقاومة في التصدي للعدو، وفشل كل المخططات والحروب للقضاء عليها أو محاصرة خيارها ونزع سلاحها من جهة، واستمرار الخطر الإسرائيلي على لبنان وعدم زوال التهديد عنه من جهةٍ أخرى يَفرض على المقاومة السعي الدؤوب لامتلاك أسباب القوة وتعزيز قدراتها وإمكاناتها بما يساعد على تأدية واجبها والقيام بمسؤولياتها الوطنية، للمساهمة في استكمال مهمة تحرير ما تبقّى من أرضنا تحت الإحتلال في مزارع شبعا وتلال كفرشوبا وبلدة الغجر اللبنانية، واستنقاذ مَن بقي من أسرى ومفقودين وأجساد الشهداء، والمشاركة في وظيفة الدفاع والحماية للأرض والشعب.ثالثاً : الدولة والنظام السياسي إنّ المشكلة الأساسية في النظام السياسي اللبناني، والتي تمنع إصلاحه وتطويره وتحديثه بشكل مستمر هي الطائفية السياسية. كما أنّ قيام النظام على أسس طائفية يشكّل عائقاً قوياً أمام تحقيق ديمقراطية صحيحة يمكن على ضوئها أن تحكم الأكثرية المنتخَبة وتعارض الأقلية المنتخَبة، ويُفتح فيها الباب لتداول سليم للسلطة بين الموالاة والمعارضة أو الإئتلافات السياسية المختلفة. ولذلك فإنّ الشرط الأساس لتطبيق ديمقراطية حقيقية من هذا النوع هو إلغاء الطائفية السياسية من النظام، وهو ما نص "اتفاق الطائف" على وجوب تشكيل هيئة وطنية عليا لإنجازه.وإلى أن يتمكن اللبنانيون ومن خلال حوارهم الوطني من تحقيق هذا الإنجاز التاريخي والحساس – نعني إلغاء الطائفية السياسية – وطالما أنّ النظام السياسي يقوم على أسس طائفية فإنّ الديمقراطية التوافقية تبقى القاعدة الأساس للحكم في لبنان، لأنها التجسيد الفعلي لروح الدستور ولجوهر ميثاق العيش المشترك. من هنا فإنّ أي مقاربة للمسائل الوطنية وفق معادلة الأكثرية والأقلية تبقى رهن تحقق الشروط التاريخية والإجتماعية لممارسة الديمقراطية الفعلية التي يصبح فيها المواطن قيمةً بحد ذاته.إنّ إرادة اللبنانيين في العيش معاً موفوري الكرامة ومتساوي الحقوق والواجبات، تحتّم التعاون البنّاء من أجل تكريس المشاركة الحقيقية والتي تشكّل الصيغة الأنسب لحماية تنوعهم واستقرارهم الكامل بعد حقبة من اللاإستقرار سببتها السياسات المختلفة القائمة على النزوع نحو الإستئثار والإلغاء والإقصاء.إنّ الديمقراطية التوافقية تشكّل صيغةً سياسيةً ملائمةً لمشاركة حقيقية من قِبَل الجميع، وعامل ثقة مطَمْئِن لمكونات الوطن، وهي تُسهم بشكل كبير في فتح الأبواب للدخول في مرحلة بناء الدولة المطَمْئِنة التي يشعر كل مواطنيها أنها قائمة من أجلهم.إنّ الدولة التي نتطلع الى المشاركة في بنائها مع بقية اللبنانيين هي:1- الدولة التي تصون الحريات العامة، وتوفر كل الأجواء الملائمة لممارستها.
2- الدولة التي تحرص على الوحدة الوطنية والتماسك الوطني.3- الدولة القادرة التي تحمي الأرض والشعب والسيادة والإستقلال، ويكون لها جيش وطني قوي ومقتدر ومجهَّز، ومؤسساتٌ أمنيةٌ فاعلةٌ وحريصةٌ على أمن الناس ومصالحهم.4- الدولة القائمة في بنيتها على قاعدة المؤسسات الحديثة والفاعلة والمتعاونة، والتي تستند الى صلاحيات ووظائف ومهام واضحة ومحددة.5- الدولة التي تلتزم تطبيق القوانين على الجميع في إطار احترام الحريات العامة والعدالة في حقوق وواجبات المواطنين، على اختلاف مذاهبهم ومناطقهم واتجاهاتهم.6- الدولة التي يتوافر فيها تمثيل نيابي سليم وصحيح لا يمكن تحقيقه إلاّ من خلال قانون انتخابات عصري يتيح للناخب اللبناني أن يختار ممثليه بعيداً عن سيطرة المال والعصبيات والضغوط المختلفة، ويحقق أوسع تمثيل ممكن لمختلف شرائح الشعب اللبناني.7- الدولة التي تعتمد على أصحاب الكفاءات العلمية والمهارات العملية وأهل النزاهة بغض النظر عن انتماءاتهم الطائفية، والتي تضع آليات فاعلةً وقويةً لتطهير الإدارة من الفساد والفاسدين دون مساومة.8- الدولة التي تتوافر فيها سلطة قضائية عليا ومستقلة وبعيدة عن تحكّم السياسيين، يمارس فيها قضاة كَفُؤون ونزيهون وأحرارٌ مسؤولياتِهم الخطيرة في إقامة العدل بين الناس.9- الدولة التي تُقِيْم اقتصادها بشكل رئيس على قاعدة القطاعات المنتِجة، وتعمل على استنهاضها وتعزيزها، وخصوصاً قطاعات الزراعة والصناعة، وإعطائها الحيّز المناسب من الخطط والبرامج والدعم بما يؤدي الى تحسين الإنتاج وتصريفه، وما يوفر فرص العمل الكافية والمناسبة وخاصةً في الأرياف.10- الدولة التي تعتمد وتطبق مبدأ الإنماء المتوازن بين المناطق، وتعمل على ردم الهوّة الإقتصادية والإجتماعية بينها.11- الدولة التي تهتم بمواطنيها، وتعمل على توفير الخدمات المناسبة لهم من التعليم والطبابة والسكن الى تأمين الحياة الكريمة، ومعالجة مشكلة الفقر، وتوفير فرص العمل وغير ذلك.12- الدولة التي تعتني بالأجيال الشابة والصاعدة، وتساعد على تنمية طاقاتِهم ومواهِبهم وتوجيههم نحو الغايات الإنسانية والوطنية، وحمايتهم من الإنحراف والرذيلة.13- الدولة التي تعمل على تعزيز دور المرأة وتطوير مشاركتها في المجالات كافةً، في إطار الإستفادة من خصوصيتها وتأثيرها واحترام مكانتِها.14- الدولة التي تُوْلي الوضع التربوي الأهمية المناسبة خصوصاً لجهة الإهتمام بالمدرسة الرسمية، وتعزيز الجامعة اللبنانية على كل صعيد، وتطبيق إلزامية التعليم الى جانب مجّانيته.15- الدولة التي تعتمد نظاماً إدارياً لا مركزياً يعطي سلطات إداريةً واسعةً للوحدات الإدارية المختلفة (محافظة/ قضاء/ بلدية)، بهدف تعزيز فرص التنمية وتسهيل شؤون ومعاملات المواطنين، دون السماح بتحوّل هذه اللامركزية الإدارية الى نوع من "الفدرلة" لاحقاً.16- الدولة التي تجهد لوقف الهجرة من الوطن، هجرة الشباب والعائلات وهجرة الكفاءات والأدمغة ضمن مخطط شامل وواقعي.17- الدولة التي ترعى مواطنيها المغتربين في كل أصقاع العالم، وتدافع عنهم وتحميهم، وتستفيد من انتشارهم ومكانتهم ومواقعهم لخدمة القضايا الوطنية.إنّ قيام دولة بهذه المواصفات والشروط هدف لنا ولكل لبناني صادق ومخلص، ونحن في حزب الله سنبذل كل جهودنا وبالتعاون مع القوى السياسية والشعبية المختلفة التي تشاركنا هذه الرؤية من أجل تحقيق هذا الهدف الوطني النبيل.رابعاً : لبنان والعلاقات اللبنانية – الفلسطينيةكواحدة من النتائج المأساوية لنشوء الكيان الصهيوني على أرض فلسطين وتشريد أهلها منها، كانت مشكلةُ اللاجئين الفلسطينيين، الذين انتقلوا إلى لبنان ليعيشوا فيه مؤقتاً كضيوف لدى إخوانهم اللبنانيين ريثما يعودون إلى وطنهم وديارهم التي أُخرجوا منها.وما عاناه الفلسطينيون واللبنانيون على حد سواء جراء هذا اللجوء، كان سببه الحقيقي والمباشر هو الإحتلال الإسرائيلي لفلسطين، وما نتج عنه من مآسٍ وويلات أصابت شعوب المنطقة بأسرها ولم يقتصر ضررها على الفلسطينيين فقط.كما أنّ معاناة اللاجئين الفلسطينيين في لبنان لم تقتصر على آلام الهجرة القسرية واللجوء فقط إنما أضيفت إليها الإعتداءات والمجازر الوحشية الإسرائيلية، التي دمرت الحجر والبشر، كما حصل في مخيم النبطية الذي دُمّر بالكامل/ قساوة العيش في المخيمات في ظل ظروف تفتقر إلى الحد الأدنى من مقومات العيش الكريم/ الحرمان من الحقوق المدنية والإجتماعية/ عدم قيام الحكومات اللبنانية المتعاقبة بواجبها تجاههم.إنّ هذا الواقع غير الطبيعي بات يحتّم اليوم على السلطات اللبنانية المسؤولة ضرورة تحمّل مسؤوليتها، وبناء العلاقات اللبنانية – الفلسطينية على أُسس صحيحة ومتينة وقانونية تراعي موازين الحق والعدل والمصالح المشتركة لكلا الشعبين، وأن لا يبقى هذا الوجود وهذه العلاقات محكومةً للأمزجة والأهواء والحسابات السياسية والتجاذبات الداخلية والتدخلات الدُّولية. إننا نرى أنّ النجاح في هذه المهمة يتحقق من خلال ما يلي:1. الحوار اللبناني – الفلسطيني المباشر.
2. تمكين الفلسطينيين في لبنان من التوافق على اختيار مرجعية موحَّدة لهم تمثلهم في هذا الحوار، متجاوزين التباينات الحاصلة في الوضع الفلسطيني الأعم.3. إعطاء الفلسطينيين في لبنان حقوقهم المدنية والإجتماعية، بما يليق بوضعهم الإنساني ويحفظ شخصيتهم وهويتهم وقضيتهم.4. التمسك بحق العودة ورفض التوطين.خامساً : لبنان والعلاقات العربيةإنّ لبنان العربي الهوية والإنتماء إنما يمارس هويته وانتماءه بوصفهما حالةً طبيعيةً أصيلةً في التكوين المجتمعي اللبناني.كما أنّ المدى الحيوي والجغرافيا السياسية والعمق الإستراتيجي وسياسات التكامل الإقليمي والمصالح القومية، بوصفها محدِّدات إستراتيجية لموقع لبنان السياسي ومصالحه الكبرى، تحتّم عليه الإلتزام بالقضايا العربية العادلة، وفي طليعتها قضية فلسطين والصراع مع العدو الإسرائيلي.كذلك، ثمة حاجةٌ أكيدةٌ إلى تضافر الجهود لتجاوز حالة الصراعات التي تشقّ الصف العربي، إذ أنّ تناقض الإستراتيجيات واختلاف التحالفات، رغم جِدّيتها وحِدَّتها، لا تبرّر الإنسياق في سياسات الإستهداف أو الإنخراط في المشاريع الخارجية القائمة على تعميق الفُرقة وإثارة النعرات الطائفية وتحريك عوامل التجزئة والتفتيت، بما يؤدي إلى إنهاك الأمة وخدمة العدو الصهيوني وتنفيذ المآرب الأميركية.إنّ تطوير المممارسة السياسية القائمة على حصر النزاعات أو تنظيمها والحؤول دون انفلاتها إلى مجال الصراعات المفتوحة، هو خيار جدير بالتبنّي لإنضاج مقاربةٍ نوعيةٍ ومسؤولةٍ في التعاطي مع القضايا القومية، وكذلك البحث عن المشتركات لتعزيزها وتوفير فرص التواصل البنّاء على مستوى الحكومات والشعوب، لتحقيق أوسع إطار تضامني يخدم قضايانا.وهنا يشكّل خيار المقاومة حاجةً جوهريةً وعاملاً موضوعياً لتصليب الموقف العربي وإضعاف العدو، بمعزل عن طبيعة الإستراتيجيات أو الرهانات السياسية المتخذة. بناءً على كل ما تقدّم لا تجد المقاومةُ غضاضةً في تعميم عوائد الإستفادة من خيار المقاومة بحيث يطال مختلفَ المواقع العربية، ما دامت النتائج تندرج في إطار معادلة إضعاف العدو وتقوية وتصليب الموقف العربي.وفي هذا الإطار، سجلت سوريا موقفاً مميزاً وصامداً في الصراع مع العدو الإسرائيلي، ودعمت حركات المقاومة في المنطقة، ووقفت الى جانبها في أصعب الظروف، وسعت الى توحيد الجهود العربية لتأمين مصالح المنطقة ومواجهة التحديات.إننا نؤكد على ضرورة التمسك بالعلاقات المميزة بين لبنان وسوريا بوصفها حاجةً سياسيةً وأمنيةً واقتصاديةً مشتركةً، تُمليها مصالح البلدين والشعبين وضرورات الجغرافيا السياسية وموجبات الإستقرار اللبناني ومواجهة التحديات المشتركة، كما ندعو الى إنهاء كل الأجواء السلبية التي شابت علاقات البلدين في السنوات القليلة الماضية والعودة بهذه العلاقات الى وضعها الطبيعي في أسرع وقت ممكن.سادساًًًً : لبنان والعلاقات الإسلاميةيواجه عالَمُنا العربي والإسلامي تحدياتٍ تطال مجتمعاتنا بمكوناتها المختلفة ما يقتضي عدم التهاون بخطورتها.فالإحتقان الطائفي والتوترات المذهبية المفتعَلة، وعلى الأخص بين السنّة والشيعة، واختلاق التناقضات القومية بين كُرد وتركمان وعرب، وإيرانيين وعرب.. وتخويف الأقليات وترهيبها، والنزف المسيحي المستمر من المشرق العربي وخاصةً من فلسطين والعراق فضلاً عن لبنان، كل ذلك يهدد تماسك مجتمعاتنا، ويقلل من منعتها، ويفاقم من عوائق نهضتها وتطورها.وبدل أن يمثِّلَ التنوعُ الديني والقومي مصدر غنًى وحيويةً إجتماعيةً فقد أسيء توظيفُه، وتم استخدامه كعامل تمزيق وفُرقة وتفتيت مجتمعي.إنّ الحالة الناجمة عن هذا الإستخدام السيئ هي حصيلة تقاطع لسياسات غربية متعمدة، وأميركية تحديداً، مع ممارسات وتصورات داخلية عصبوية لا مسؤولة، بالإضافة إلى بيئة سياسية غير مستقرة.إنّ أخذ هذه الحقائق بعين الإعتبار يبدو ملحّاً، ومن الجدير والضروري إدراجها كأحد الإهتمامات الجوهرية في برامج القوى والإتجاهات الأساسية بما فيها الحركات الإسلامية التي تقع على كاهلها مسؤولية خاصة في التصدي لهذه التحديات ومعالجة تلك المشكلات.يؤكد حزب الله على أهمية التعاون بين الدول الإسلامية في المجالات كافةً، وهو ما يمنحها قوة تضامن في وجه المخططات الإستكبارية، وحمايةً مجتمعيةً من الغزو الثقافي والإعلامي، ويحضّها على الإستفادة من خيراتها في تبادل المنافع المختلفة بين هذه الدول.وفي هذا الإطار يَعتبر حزب الله إيران الإسلام دولةً مركزيةً مهمةً في العالم الإسلامي، فهي التي أسقطت بثورتها نظام الشاه ومشاريعه الصهيونية – الأميركية، ودعمت حركات المقاومة في منطقتنا، ووقفت بشجاعة وتصميم الى جانب القضايا العربية والإسلامية وعلى رأسها القضية الفلسطينية.إنّ سياسة الجمهورية الإسلامية في إيران واضحةٌ وثابتةٌ في دعم القضية المركزية الأُولى والأهم للعرب والمسلمين وهي القضية الفلسطينية، منذ إعلان انتصار الثورة الإسلامية المباركة بقيادة الولي الفقيه الإمام الخميني (قدس سره)، وفتح أول سفارة فلسطينية مكان السفارة الإسرائيلية، وقد استمر هذا الدعم بأشكاله كافةً الى يومنا هذا بقيادة الولي الفقيه الإمام الخامنئي (دام ظلّه)، ما أدى الى تحقيق انتصاراتٍ بارزةٍ لأول مرة في تاريخ الصراع مع الصهاينة الغزاة.إنّ اختلاق التناقض مع الجمهورية الإسلامية في إيران من قِبَل بعض الجهات العربية يمثّل طعناً للذات وللقضايا العربية، ولا يخدم سوى "إسرائيل" والولايات المتحدة الأميركية.فإيران التي صاغت عقيدتها السياسية، وبَنَتْ مداها الحيوي على قاعدة "مركزية القضية الفلسطينية" والعداء لـ"إسرائيل" ومواجهة السياسات الأميركية والتكامل مع البيئة العربية والإسلامية، يجب أن تُقابَل بإرادة التعاون والأخوّة، والتعاطي معها كقاعدة استنهاض ومركز ثقل استراتيجي وأنموذج سيادي واستقلالي وتحرري داعم للمشروع العربي – الإسلامي الإستقلالي المعاصر، وقوةً تزيد دول وشعوب منطقتنا قوةً ومنعةً.إنّ العالم الإسلامي يَقْوى بتحالفاته وتعاون دوله. ونؤكد على أهمية الإستفادة من عناصر القوة السياسية والإقتصادية والبشرية.. الموجودة في كل دولة من دول العالم الإسلامي، على قاعدة التكامل والنصرة وعدم التبعية للمستكبرين.ونذكّر بأهمية الوحدة بين المسلمين، قال تعالى: {وَاعْتَصِمُواْ بِحَبْلِ اللّهِ جَمِيعاً وَلاَ تَفَرَّقُواْ}، والحذر مما يسبب التفرقة بينهم كالإثارات المذهبية وخاصةً بين السنّة والشيعة، ونحن نراهن على وعي الشعوب الإسلامية في التصدي لما يحاك من مؤامرات وفتن على هذا الصعيد.سابعاً : لبنان والعلاقات الدُّوليةإنّ معايير الإختلاف والنزاع والصراع في رؤية حزب الله ومنهجه إنما تقوم على أساس سياسي – أخلاقي بالدرجة الأولى، بين مستكبِر ومستضعَف، وبين متسلط ومقهور، وبين متجبّر محتل وطالب حرية واستقلال.كما يَعتبر حزب الله أنّ الهيمنة الأُحادية تُطيح بالتوازن والإستقرار العالميَين وبالأمن والسلم الدُّوليَين.إنّ دعم الإدارة الأميركية اللامحدود لـ"إسرائيل" ودفعها للعدوان وتغطية احتلالاتها للأراضي العربية، بالإضافة إلى هيمنة الإدارة الأميركية على المؤسسات الدُّولية، وازدواجية المعايير في إصدار وتنفيذ القرارات الدُّولية، وسياسة التدخل في شؤون المجتمعات الأخرى، وعسكرة العالم واعتماد منطق الحروب المتنقلة في النزاعات الدُّولية، وإثارة القلاقل والإضطرابات في كل أنحاء العالم، يضع الإدارة الأميركية في موقع المعادي لأمتنا وشعوبنا، كما يحمّلها مسؤوليةً أساسيةً وأُولى في إنتاج الإختلال والإضطراب في النظام الدُّولي.أما السياسات الأوروبية فإنها تتأرجح بين العجز وقلة الفاعلية من ناحية والإلتحاق – غير المبرَّر – بالسياسات الأميركية من ناحية ثانية، ما يؤدي فعلياً إلى تجويف النزعة المتوسطية في أوروبا لصالح هيمنة النزعة الأطلسية، بخلفياتها الإستعمارية.إنّ الإلتحاق بالسياسات الأميركية – وخاصةً في مرحلة إخفاقها التاريخي – يشكّل خطأً استراتيجياً لن يؤدي إلاّ إلى مزيد من المشكلات والتعثر والتعقيدات في العلاقات الأوروبية – العربية.إنّ على أوروبا مسؤوليةً خاصةً بفعل الإرث الإستعماري الذي ألحق بمنطقتنا أضراراً فادحةً لا تزال شعوبنا تعاني من نتائجها وتأثيراتها.ولأنّ شعوباً أوروبيةً لها تاريخ في مقاومة المحتل فإنّ واجب أوروبا الأخلاقي والإنساني – قبل السياسي – يفرض عليها الإعتراف بحق الشعوب في مقاومة المحتل، على قاعدة التمييز بين المقاومة والإرهاب.وبحسب رأينا: إنّ مقتضيات الإستقرار والتعاون في العلاقات الأوروبية – العربية تستوجب بناء مقاربة أوروبية أكثر استقلاليةً وأكثر عدالةً وموضوعيةً. وسيكون متعذراً بناء المدى الحيوي المشترك، سياسياً وأمنياً، من دون هذا التحول الكفيل بمعالجة مَواطن الخلل المولِّدة للأزمات واللاإستقرار.ومن ناحية أخرى ننظر بكثير من الإهتمام والتقدير للتجربة الإستقلالية والتحررية الرافضة للهيمنة في دول أميركا اللاتينية، ونرى مساحات واسعةً من التلاقي بين مشروعها ومشروع حركات المقاومة في منطقتنا، بما يُفضي إلى بناء نظام دُوليّ أكثر عدالةً وتوازناً.إنّ ملاقاة تلك التجربة تشكّل باعثاً لآمال واعدة على المستوى العالمي، بالإستناد إلى هوية إنسانية جامعة وخلفية سياسية وأخلاقية مشتركة. وفي هذا السياق سيبقى شعار "وحدة المستضعفين" أحد مرتكزات فكرنا السياسي في بناء فهمنا وعلاقاتنا ومواقفنا تجاه القضايا الدُّولية.(الفصل الثالث)  فلسطين ومفاوضات التسويةأولاً : قضية فلسطين والكيان الصهيونيشكّل الكيان الصهيوني منذ اغتصابه لفلسطين وتشريد أهلها منها في العام 1948، برعاية ودعم من قوى الهيمنة الدُّولية آنذاك، عدواناً مباشراً وخطراً جِدّياً طالا المنطقة العربية بأكملها، وتهديداً حقيقياً لأمنها واستقرارها ومصالحها، ولم يقتصر أذاه وضرره على الشعب الفلسطيني أو الدول والشعوب المجاورة لفلسطين فحسب، وما الإعتداءات والتوترات والحروب التي شهدتها المنطقة بفعل النزعة والممارسات العدوانية الإسرائيلية إلاّ الدليل والشاهد على مقدار الظلم الذي لحق بالشعب الفلسطيني وبالعرب والمسلمين جراء الجريمة ضد الإنسانية التي ارتكبها الغرب عندما أقدم على زرع هذا الكيان الغريب في قلب العالم العربي والإسلامي، ليكون اختراقاً معادياً وموقعاً متقدماً للمشروع الإستكباري الغربي عامةً، وقاعدةً للسيطرة والهيمنة على المنطقة خاصةً.إنّ الحركة الصهيونية هي حركة عنصرية فكراً وممارسةً، وهي نتاج عقلية إستكبارية إستبدادية تسلطية، ومشروعها في أصله وأساسه هو مشروع إستيطاني تهويدي توسعي. كما أنّ الكيان الذي انبثق عنها قام وتمكّن واستمر عبر الإحتلال والعدوان والمجازر والإرهاب، بدعم ورعاية واحتضان من الدول الإستعمارية، لا سيما الولايات المتحدة الأميركية، التي ترتبط معه بتحالف إستراتيجي جعلها شريكاً حقيقياً له في كل حروبه ومجازره وممارساته الإرهابية.إنّ الصراع الذي نخوضه وتخوضه أمتنا ضد المشروع الصهيوني – الإستعماري في فلسطين إنما هو قيامٌ بواجب الدفاع عن النفس ضد الإحتلال والعدوان والظلم الإسرائيلي – الإستكباري الذي يتهدد وجودنا ويستهدف حقوقنا ومستقبلنا، وهو ليس قائماً على المواجهة الدينية أو العنصرية أو العرقية من جانبنا، وإنْ كان أصحاب هذا المشروع الصهيوني – الإستعماري لم يتورعوا يوماً عن استخدام الدين وتوظيف المشاعر الدينية وسيلةً لتحقيق أهدافهم وغاياتهم.وليس ما ذهب إليه الرئيس الأميركي "بوش" وخَلَفُه "أوباما" وقادة الكيان الصهيوني معهما، من مطالبة للفلسطينيين والعرب والمسلمين بالإعتراف بيهودية "دولة إسرائيل" إلاّ أوضح دليل على ذلك.إنّ النتيجة الطبيعية والحتمية أن يعيش هذا الكيان الغاصب المفتعَل مأزقاً وجودياً يؤرِّق قادتَه وداعميه، لكونه مولوداً غير طبيعي وكِياناً غير قابل للحياة والإستمرار ومعرَّضاً للزوال. وهنا تقع المسؤولية التاريخية على عاتق الأمة وشعوبها أن لا تعترف بهذا الكيان مهما كانت الضغوطات والتحديات، وأن تواصل العمل من أجل تحرير كل الأرض المغتصبة واستعادة كل الحقوق المسلوبة مهما طال الزمن وعظمت التضحيات.ثانياً : القدس والمسجد الأقصىيدرك العالم بأسره مكانة وقداسة مدينة القدس والمسجد الأقصى، فالمسجد الأقصى هو أُولى القِبلتين وثالث الحرمين ومسرى رسول الله (صلّى الله عليه وعلى آله وسلَّم)، وملتقى الأنبياء والرسل (عليهم صلوات الله أجمعين)، ولا يُنكِر أحدٌ عظيمَ مكانته لدى المسلمين كمَعْلَمٍ من أكثر المعالم قدسيةً عندهم، وعمقَ علاقته بالإسلام كواحد من أهم الرموز الإسلامية على وجه الأرض.
ومدينة القدس بما تحتضن من مقدسات إسلامية ومسيحية، تتمتع بمكانة رفيعة لدى المسلمين والمسيحيين على حد سواء.إنّ استمرار الإحتلال الإسرائيلي لهذه المدينة المقدسة مع ما يرافق ذلك من خطط ومشاريع تهويدية وطرد أبنائها ومصادرة بيوتهم وممتلكاتهم وإحاطتها بأحياء يهودية وأحزمة وكتل إستيطانية وخنقها بجدار الفصل العنصري، بالإضافة إلى المساعي الأميركية – الإسرائيلية المتواصلة لتكريسها عاصمةً أبديةً للكيان الصهيوني باعتراف دُوليّ، كلها إجراءات عدوانية مرفوضة ومدانة.كما أنّ الإعتداءات الخطيرة المتواصلة والمتكررة على المسجد الأقصى المبارك وما ينفَّذ في نطاقه من حفريات وما يُعَدّ من خطط لتدميره، تشكّل خطراً جدياً وحقيقياً يهدد وجودَه وبقاءه ويُنذر بتداعيات خطيرة على المنطقة بأسرها.إنّ واجب نُصرة القدس وتحريرها والدفاع عن المسجد الأقصى وحمايته، هو واجبٌ ديني ومسؤوليةٌ إنسانية وأخلاقية في عنق كل حرّ وشريف من أبناء أمتنا العربية والإسلامية وكل أحرار وشرفاء العالم.إننا ندعو ونطالب العرب والمسلمين على الصعيدين الرسمي والشعبي، وجميع الدول الحريصة على السلام والإستقرار في العالم، لبذل الجهود والإمكانيات لتحرير القدس من نير الإحتلال الصهيوني، وللمحافظة على هويتها الحقيقية ومقدساتها الإسلامية والمسيحية.ثالثاً : المقاومة الفلسطينيةإنّ الشعب الفلسطيني وهو يخوض معركة الدفاع عن النفس ويكافح لاستعادة حقوقه الوطنية المشروعة في فلسطين بمعناها التاريخي وواقعها الجغرافي إنما يمارس حقاً مشروعاً تُقِرّه وتُوجِبه الرسالات السماوية والقوانين الدُّولية والقيم والأعراف الإنسانية.وهذا الحق يشمل المقاومة بكل أشكالها – وفي مقدمتها الكفاح المسلح – وبكل الوسائل التي تتمكن فصائل المقاومة الفلسطينية من استخدامها، خاصةً في ظل اختلال موازين القوى لمصلحة العدو الصهيوني المتسلح بأحدث أسلحة الفتك والدمار والقتل.ولقد أثبتت التجارب – التي شكلت دليلاً قطعياً لا يدع مجالاً للشك والإرتياب على امتداد مسيرة الصراع والمواجهة بين أمتنا وبين الكيان الصهيوني منذ اغتصابه لفلسطين وحتى يومنا هذا – أهمية وجدوى خيار المقاومة الجهادية والكفاح المسلح في مواجهة العدوان وتحرير الأرض واستعادة الحقوق وتحقيق توازن الرعب وسد فجوة التفوق الإستراتيجي عبر المعادلات التي فرضتها المقاومة بإمكانياتها المتاحة وإرادتها وعزيمتها في ميدان المواجهة، وخير شاهد ودليل على ذلك ما حققته المقاومة في لبنان من انتصارات متتالية، وما راكمته من إنجازات ميدانية وعسكرية ومعنوية على امتداد تجربتها الجهادية، لا سيما عبر إرغام الصهاينة على الإنسحاب الإسرائيلي الكبير في أيار العام 2000 من معظم الأراضي اللبنانية المحتلة، أو عبر الفشل المدوي للجيش الصهيوني في عدوان تموز العام 2006، والذي حققت فيه المقاومة انتصاراً إلهياً وتاريخياً واستراتيجياً غيّر معادلة الصراع بشكل جذري، وألحق أول هزيمة بهذا المستوى بالعدو الإسرائيلي، وأسقط أسطورة الجيش الذي لا يُقهر.والدليل الآخر على ذلك هو ما حققته المقاومة في فلسطين من إنجازات متواصلة عبر تجربة الثورة الفلسطينية وخيار الكفاح المسلح الذي انتهجته، وعبر انتفاضة الحجارة الأولى وانتفاضة الأقصى الثانية، وصولاً إلى الإندحار القهري للجيش الإسرائيلي عبر الإنسحاب الكامل من قطاع غزة في العام 2005 بلا قيد أو شرط وبلا تفاوض أو اتفاق، ومن دون تحقيق أي مكسب سياسي أو أمني أو جغرافي، ليكون ذلك أول انتصار ميداني – جغرافي – نوعي بهذا الحجم وهذا المستوى وهذه الدلالة لخيار المقاومة في فلسطين، كونه أولَ انسحاب إسرائيلي إضطراري بفعل المقاومة، ضمن حدود فلسطين التاريخية، والدلالات التي يحملها هذا الأمر بالغة الأهمية في مجرى الصراع بيننا وبين الكيان الصهيوني على الصعيد الإستراتيجي. كما أنّ الصمود الرائع للشعب الفلسطيني المجاهد ومقاومته في غزة في مواجهة العدوان الصهيوني سنة 2008 درس للأجيال وعبرة للغزاة والمعتدين.فإذا كانت هذه هي جدوى المقاومة في لبنان وفي فلسطين، فماذا كانت جدوى الخيار التفاوضي التسووي؟ وما هي النتائج والمصالح والمكاسب التي حققتها المفاوضات في كل مراحلها وعبر كل الإتفاقات التي أنتجتها؟ أليس المزيد من الغطرسة والتسلط والتعنت الإسرائيلي والمزيد من المكاسب والمصالح والشروط الإسرائيلية؟إننا إذ نؤكد وقوفنا الدائم والثابت إلى جانب الشعب الفلسطيني والقضية الفلسطينية، بثوابتها التاريخية والجغرافية والسياسية، نؤكد بشكل قاطع وجازم مساندتنا وتأييدنا ودعمنا لهذا الشعب وحركات المقاومة الفلسطينية ونضالها في مواجهة المشروع الإسرائيلي.رابعاً : مفاوضات التسويـةكان موقفنا ولا يزال وسيبقى تجاه عملية التسوية وتجاه الإتفاقات التي أنتجها مسار مدريد التفاوضي عبر "اتفاق وادي عربة" وملحقاته و"اتفاق أوسلو" وملحقاته ومن قبلهما "اتفاق كامب ديفيد" وملحقاته، موقفَ الرفض المطلق لأصل ومبدأ خيار التسوية مع الكيان الصهيوني، القائم على أساس الإعتراف بشرعية وجوده، والتنازل له عما اغتصبه من أرض فلسطين العربية والإسلامية.هذا الموقف هو موقف ثابت ودائم ونهائي، غير خاضع للتراجع أو المساومة، حتى لو اعترف العالم كله بـ"إسرائيل".ومن هذا المنطلق ومن موقع الأخوّة والمسؤولية والحرص، فإننا ندعو المسؤولين العرب إلى أن يلتزموا خيارات شعوبهم عبر إعادة النظر بالخيار التفاوضي وإجراء مراجعة لنتائج الإتفاقات الموقَّعة مع العدو الصهيوني، والتخلي الحاسم والنهائي عن عملية التسوية الوهمية الظالمة المسمّاة زوراً وبهتاناً "عملية السلام"، لا سيما وأنّ مَن راهنوا على دور للإدارة الأميركية المتعاقبة كشريك ووسيط نزيه وعادل في هذه العملية، قد عاينوا بما لا يقبل الشك أنها خذلتهم، ومارست عليهم الضغط والإبتزاز، وأظهرت العداء لشعوبهم وقضاياهم ومصالحهم، وانحازت بشكل كامل وسافر إلى جانب حليفها الإستراتيجي الكيان الصهيوني.أما الكيان الصهيوني، الذي يتوهمون إمكانية إقامة سلام معه، فقد أظهر لهم في كل مراحل المفاوضات أنه لا يطلب السلام ولا يسعى إليه، وأنه يستخدم المفاوضات لفرض شروطه وتعزيز موقعه وتحقيق مصالحه وكسر حدّة العداء والحاجز النفسي لدى شعوبهم تجاهه، عبر حصوله على تطبيع رسمي وشعبي مجاني ومفتوح، يحقق له التعايش الطبيعي والإندماج في النظام الإقليمي وفرض نفسه كأمر واقعي في المنطقة والقبول به والإعتراف بشرعية وجوده، بعد التخلي له عن الأرض الفلسطينية التي اغتصبها.من هنا فإننا ندعو ونتوقع ونأمل من كل العرب والمسلمين على الصعيدين الرسمي والشعبي العودةَ إلى فلسطين والقدس كقضية مركزية لهم جميعاً، يتوحدون حولها ويلتزمون تحريرها من رجس الإحتلال الصهيوني الغاشم، والقيامَ بما يمليه عليهم واجبُهم الديني والأخوي والإنساني تجاه مقدساتهم في فلسطين وتجاه شعبها المظلوم، وتوفيرَ كل مستلزمات الدعم لتعزيز صمود الشعب الفلسطيني وتمكينه من مواصلة مقاومته، ورفضَ كل مشاريع التطبيع مع العدو الصهيوني وإسقاطها، والتمسكَ بحق عودة جميع اللاجئين الفلسطينيين إلى أراضيهم وديارهم التي أُخرجوا منها، والرفضَ القاطع لكل البدائل المطروحة من توطين أو تعويض أو تهجير، والعملَ الفوري على فك الحصار المفروض على الشعب الفلسطيني لا سيما الحصار الشامل لقطاع غزة، وتبنّي قضية أكثر من أحد عشر ألف أسير ومعتقل في السجون الإسرائيلية، ووضعَ الخطط والبرامج العملية لتحريرهم من الأسر.

الخــاتمــة

هذه هي رؤيتنا وتصوراتنا، حَرِصْنا في البحث عنها أن نكون طلاب حق وحقيقة. وهذه هي مواقفنا والتزاماتنا، سعينا أن نكون فيها أهلَ صدق ووفاء، نؤمن بالحق وننطق به وندافع عنه ونضحي من أجله حتى الشهادة، لا نبغي في ذلك سوى رضا خالقنا وإلهنا رب السماوات والأرض، ولا نرجو من ذلك سوى صلاح أهلنا وشعبنا وأمتنا وخيرهم وسعادتهم في الدنيا والآخرة.اللهم إنك تعلم أنه لم يكن الذي كان منا منافسةً في سلطان ولا ابتغاءً لشيء من الحُطام، وإنما كان إحياءً للحق وإماتةً للباطل ودفاعاً عن مظلومي عبادك وإقامةً للعدل في أرضك وطلباً لرضاك والقرب منك، على هذا قضى شهداؤنا، وعلى هذا نمضي ونواصل العمل والجهاد، وقد وعدتنا إحدى الحُسنيين إما النصر أو التشرّف بلقائك مخضّبين بدمائنا.ووَعْدُنا لك يا رب، ولكل عبادك المظلومين، أن نبقى الرجالَ الصادقين في العهد، والمنتظرين للوعد، والثابتين الذين ما بدّلوا تبديلاً.

المصدر: الجزيرة"""

In [6]:
# Example: turn Arabic text into a list of sentences
import re

def arabic_text_to_sentences(text):
    # Split by Arabic/standard punctuation that indicates sentence endings
    sentences = re.split(r'(?<=[.!؟…])\s+', text)
    return [sentence.strip() for sentence in sentences if sentence.strip()]


In [7]:
def split_text_arabic(text, max_chunk_size=2000):
    import re
    sentences = re.split(r'(?<=[.!؟…])\s*', text)
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) < max_chunk_size:
            current_chunk += sentence + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
    if current_chunk.strip():
        chunks.append(current_chunk.strip())
    return chunks

In [8]:
def translate_long_text(text, client, source='ar', target='en'):
    chunks = split_text_arabic(text)
    translated_chunks = []

    for i, chunk in enumerate(chunks):
        print(f"🔄 Translating chunk {i+1}/{len(chunks)}...")
        translated = client.translate(chunk, source_language=source, target_language=target)['translatedText']
        translated_chunks.append(translated)
    
    return "\n\n".join(translated_chunks)

In [9]:
translated_title_long = translate_long_text(arabic_title, client)
translated_body_long = translate_long_text(arabic_text, client)

🔄 Translating chunk 1/1...
🔄 Translating chunk 1/23...
🔄 Translating chunk 2/23...
🔄 Translating chunk 3/23...
🔄 Translating chunk 4/23...
🔄 Translating chunk 5/23...
🔄 Translating chunk 6/23...
🔄 Translating chunk 7/23...
🔄 Translating chunk 8/23...
🔄 Translating chunk 9/23...
🔄 Translating chunk 10/23...
🔄 Translating chunk 11/23...
🔄 Translating chunk 12/23...
🔄 Translating chunk 13/23...
🔄 Translating chunk 14/23...
🔄 Translating chunk 15/23...
🔄 Translating chunk 16/23...
🔄 Translating chunk 17/23...
🔄 Translating chunk 18/23...
🔄 Translating chunk 19/23...
🔄 Translating chunk 20/23...
🔄 Translating chunk 21/23...
🔄 Translating chunk 22/23...
🔄 Translating chunk 23/23...


In [19]:
with open("translated_article.txt", "w", encoding="utf-8") as f:
    f.write(translated_title_long + "\n\n" + translated_body_long)

print("✅ Saved as translated_article.txt")


✅ Saved as translated_article.txt


In [20]:
# If not, download it from https://dejavu-fonts.github.io/ and place it in your folder.

filename = "Manifesto_2009.pdf"

pdf = FPDF()
pdf.add_page()
pdf.set_auto_page_break(auto=True, margin=15)

# Add Unicode font
pdf.add_font('DejaVu', '', 'dejavu-fonts-ttf-2.37/ttf/DejaVuSans.ttf', uni=True)
pdf.add_font('DejaVu', 'B', 'dejavu-fonts-ttf-2.37/ttf/DejaVuSans-Bold.ttf', uni=True)
pdf.add_font('DejaVu', 'I', 'dejavu-fonts-ttf-2.37/ttf/DejaVuSans-Oblique.ttf', uni=True)
# ...existing code...e=14)
pdf.set_font('DejaVu', 'B', size=14)
pdf.multi_cell(0, 10, translated_title_long)
pdf.ln()

pdf.set_font('DejaVu', size=12)
pdf.multi_cell(0, 10, translated_body_long)

pdf.output(filename)
print(f"✅ Saved as {filename}")

✅ Saved as Manifesto_2009.pdf
